* > # EE226 - Coding 2
## Streaming algorithm & Locality Sensitive Hashing

### Streaming: DGIM

DGIM is an efficient algorithm in processing large streams. When it's infeasible to store the flowing binary stream, DGIM can estimate the number of 1-bits in the window. In this coding, you're given the *stream_data.txt* (binary stream), and you need to implement the DGIM algorithm to count the number of 1-bits. Write code and ask the problems below.

### Your task

1. Set the window size to 1000, and count the number of 1-bits in the current window.

In [ ]:
import time
size_window=1000
time_location=30000
n_max_bucket=4
bucket_n=[]

In [ ]:
def Merge1():
    for i in range(len(bucket_n)-1,n_max_bucket-1,-1):
        if bucket_n[i]['sum']==bucket_n[i-n_max_bucket]['sum']:
            bucket_n[i-n_max_bucket]['sum']+=bucket_n[i-n_max_bucket+1]['sum']
            bucket_n[i-n_max_bucket]['timestamp']=bucket_n[i-n_max_bucket+1]['timestamp']
            del bucket_n[i-n_max_bucket+1]

In [ ]:
def Count_DGIM1():
    summ=0
    start_time=time.time()
    with open('../input/coding2/stream_data.txt', 'r') as f:
        temp=f.readline()
        temp=temp.replace('\t', '')
        for i in range(time_location):
            text=temp[i]
            if len(bucket_n)>0 and i+1-size_window==bucket_n[0]['timestamp']:
                del bucket_n[0]
            if int(text)==1:
                bucket={"timestamp":i+1,"sum":1}
                bucket_n.append(bucket)
                Merge1()
    for i in range(len(bucket_n)):
        summ+=bucket_n[i]['sum']
    print(bucket_n)
    summ-=bucket_n[0]['sum']/2
    return summ if len(bucket_n)>0 else 0,time.time()-start_time

In [ ]:
bit_sum,bit_time=Count_DGIM1()
print("当前窗口中1的估计个数为：%d,运行时间为:%f"%(bit_sum,bit_time))
print("相同大小桶个数越多越接近精确解")

2. Write a function that accurately counts the number of 1-bits in the current window, and compare the difference between its running time and space and the DGIM algorithm.

In [ ]:
def Count_ACTUAL():
    bit_sum=0
    start_time=time.time()
    with open('../input/coding2/stream_data.txt', 'r') as f:
        temp=f.readline()
        temp=temp.replace('\t', '')
        for i in range(time_location-size_window,time_location):
            text=temp[i]
            if int(text)==1:
                bit_sum+=1
    return bit_sum ,time.time()-start_time
                

In [ ]:
bit_act_sum,bit_act_time=Count_ACTUAL()

print("当前窗口中1的精确个数为：%d,运行时间为:%f"%(bit_act_sum,bit_act_time))

### Locality Sensitive Hashing

The locality sensitive hashing (LSH) algorithm is efficient in near-duplicate document detection. In this coding, you're given the *docs_for_lsh.csv*, where the documents are processed into set of k-shingles (k = 8, 9, 10). *docs_for_lsh.csv* contains 201 columns, where column 'doc_id' represents the unique id of each document, and from column '0' to column '199', each column represents a unique shingle. If a document contains a shingle ordered with **i**, then the corresponding row will have value 1 in column **'i'**, otherwise it's 0. You need to implement the LSH algorithm and ask the problems below.

### Your task

Use minhash algoirthm to create signature of each document, and find 'the most similar' documents under Jaccard similarity. 
Parameters you need to determine:
1) Length of signature (number of distinct minhash functions) *n*. Recommanded value: n > 20.

2) Number of bands that divide the signature matrix *b*. Recommanded value: b > n // 10.

In [ ]:
# Your code here, you can add cells if necessary
import random
import hashlib
import pandas as pd
import numpy as np
df=pd.read_csv('../input/coding2/docs_for_lsh.csv')
tmp=df.values
matrix=tmp[:,1:]
print(matrix)

In [ ]:
doc_id = df['doc_id'].tolist()
nonZeroShingle = matrix[0].nonzero()
i = 1
final_id=[0]
print(matrix.shape[0])
while i < matrix.shape[0]:
    if sum(matrix[i][nonZeroShingle]) == 0:
        i += 1
    else:   
        final_id.append(i)
        i += 1
file = []
for i in range(len(final_id)):
    file.append(matrix[final_id[i],:].tolist())

In [ ]:
matrixnew=np.array(file)
print(type(matrixnew))
print(matrixnew.shape)
matrixnew=matrixnew.T

In [ ]:
def sigGen(matrix):
    # the row sequence set
    seqSet = [i for i in range(matrix.shape[0])]
    # initialize the sig vector as [-1, -1, ..., -1]
    result = [-1 for i in range(matrix.shape[1])]
    count = 0
    while len(seqSet) > 0:
        # choose a row of matrix randomly
        randomSeq = random.choice(seqSet)
        for i in range(matrix.shape[1]):
            if matrix[randomSeq][i] != 0 and result[i] == -1:
                result[i] = randomSeq
                count += 1
        if count == matrix.shape[1]:
            break
        seqSet.remove(randomSeq)
    # return a list
    return result

In [ ]:
def sigMatrixGen(input_matrix, n):
    result = []
    for i in range(n):
        #print(i)
        sig = sigGen(input_matrix)
        result.append(sig)
    # return a ndarray
    return np.array(result)

In [ ]:
'''
result=sigMatrixGen(matrixnew, 100)
print(result)
data=[]
data1=[]
for i in range(result.shape[1]):
    data.append(len(set(result[:,0]) & set(result[:,i])))
    a=result[:,0]- result[:,i]
    summ=0
    for i in range(a.shape[0]):
        if a[i]==0:
            summ=summ+1
    data1.append(summ)
#print(data)
#print(data1)
'''

In [ ]:
# generate the sig matrix
sigMatrix = sigMatrixGen(matrixnew, 120)

In [ ]:
#直接比较相同shingle个数
data=[]
data1=[]
for i in range(sigMatrix.shape[1]):
    data.append(len(set(sigMatrix[:,0]) & set(sigMatrix[:,i])))
    a=sigMatrix[:,0]- sigMatrix[:,i]
    summ=0
    for i in range(a.shape[0]):
        if a[i]==0:
            summ=summ+1
    data1.append(summ)
rank = [index for index, value in sorted(list(enumerate(data1)), key=lambda x:x[1],reverse = True)]
print(rank[1:31])


In [ ]:
#LSH计算
b=8
r=15
hashBuckets = {}
begin, end = 0, r


# count the number of band level
count = 0

while end <= sigMatrix.shape[0]:

    count += 1

    # traverse the column of sig matrix
    for colNum in range(sigMatrix.shape[1]):

        # generate the hash object, we used md5
        hashObj = hashlib.md5()

        # calculate the hash value
        band = str(sigMatrix[begin: begin + r, colNum]) + str(count)
        hashObj.update(band.encode())

        # use hash value as bucket tag
        tag = hashObj.hexdigest()

        # update the dictionary
        if tag not in hashBuckets:
            hashBuckets[tag] = [colNum]
        elif colNum not in hashBuckets[tag]:
            hashBuckets[tag].append(colNum)
    begin += r
    end += r

In [ ]:
result_final = set()
queryCol = 0
for key in hashBuckets:
    if 0 in hashBuckets[key]:
        for i in hashBuckets[key]:
            result_final.add(i)

result_final.remove(queryCol)
print(len(result_final))
#print(hashBuckets)

In [ ]:
final=[]
result_final1=list(result_final)[0:30]
for i in result_final1:
    final.append(final_id[i])
print(final)

Problem: For document 0 (the one with id '0'), list the **30** most similar document ids (except document 0 itself). You can valid your results with the [sklearn.metrics.jaccard_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html) function.

Tips: You can adjust your parameters to hash the documents with similarity *s > 0.8* into the same bucket.

In [ ]:
# 内置函数计算
from sklearn.metrics import jaccard_score
jac_file = []
matrix1=matrixnew.T
print(matrix1)
for i_file in range(1,len(matrix1)):
    jac_file.append(jaccard_score(np.array(matrix1[0]),np.array(matrix1[i_file])))
jac_30 = []
for num in range(30):
    jac_30.append(jac_file.index(max(jac_file))+1)
    jac_file[jac_file.index(max(jac_file))] = 0
print(jac_30)

In [ ]:
final1=[]
for i in jac_30:
    final1.append(final_id[i])
print(final1)
final2=[]
for i in rank[1:31]:
    final2.append(final_id[i])


In [ ]:
#print('使用LSH得到的结果与内置函数相同的个数：%d'%len(set(final)&set(final1)))
#print('使用直接计算相似度得到的结果与内置函数相同的个数：%d'%len(set(final1)&set(final2)))